<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e874535625b901d6fa7933049ec66ce82586e6ddbaccf72c4f29d975ef761c40
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 10:02:05
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.41 C
-------------------
Today PnL: 99.78 K (0.71%)
Current PnL: -17.87 L (-12.02%)
CY Booked + Current PnL: -6.84 L (-4.6%)
-------------------
Total profit:  1.92 L
Total loss:  -19.79 L
-------------------
Total Booked + Current PnL: 20.14 L (16.55%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.8%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.52 L (54.84%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.42,61.0,H-LC,11.16,173030.0,16853.0,9240.0,0.50,10.79,5.34,16.71,22.0,1.82,1.33,30.11,X40N,BTT,HEALTHCARE
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.58,88163.0,-12614.0,12704.0,-0.84,-12.52,14.41,0.09,245.0,-0.99,0.68,15.83,OX40N,NTT,DURABLES
62,SBIN,863.00,-13.25,52.0,M-LC,4.24,215472.0,13992.0,13230.0,0.53,6.94,6.14,13.51,60.0,1.06,1.65,20.42,XY25,NTT,BANKS
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.82,110584.0,10080.0,20558.0,0.98,10.03,18.59,30.49,79.0,0.49,0.85,45.68,MH,ATH,METALS
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200246.0,108.0,20786.0,-0.13,0.05,10.38,10.44,4.0,0.01,1.54,5.62,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,41.94,62.0,M-MC,7.07,224480.0,-482.0,168136.0,0.70,-0.21,74.90,74.53,192.0,-0.00,1.72,31.97,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-20.88,42.0,H-MC,2.00,136233.0,445.0,42314.0,0.78,0.33,31.06,31.48,107.0,0.01,1.05,12.49,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,63.0,H-MC,1.73,248882.0,1151.0,73918.0,0.52,0.46,29.70,30.30,92.0,0.02,1.91,13.82,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.34,43.0,H-MC,3.49,136848.0,715.0,30408.0,0.78,0.52,22.22,22.86,91.0,0.02,1.05,13.20,X40,ATH,INSURANCE
51,PGHH,17973.08,-30.94,51.0,H-MC,4.70,199770.0,-1050.0,69820.0,-0.53,-0.52,34.95,34.25,80.0,-0.02,1.53,4.65,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.07,45.0,H-LC,13.77,218785.0,1379.0,52115.0,1.49,0.63,23.82,24.61,48.0,0.03,1.68,55.80,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.09,47.0,H-LC,8.25,172907.0,-8852.0,44109.0,0.42,-4.87,25.51,19.39,86.0,-0.20,1.33,2.88,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,47.0,H-LC,10.17,229487.0,-12437.0,75708.0,2.42,-5.14,32.99,26.15,8.0,-0.16,1.76,8.45,X40,ATH,IT
1,ABB,7934.00,-40.74,48.0,H-LC,12.54,247656.0,-13963.0,133165.0,0.54,-5.34,53.77,45.57,7.0,-0.10,1.90,5.38,AR,NTT,ELECTRICAL


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.20,-5.78,52.0,H-LC,1.60,233348.0,-12803.0,92009.0,1.90,-5.20,39.43,32.18,16.0,-0.14,1.79,28.97,X200,ATH,IT
50,NESTLEIND,1377.00,-8.22,60.0,H-LC,2.31,282274.0,16848.0,39942.0,0.34,6.35,14.15,21.40,11.0,0.42,2.17,13.41,XY25,NTT,FMCG
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200246.0,108.0,20786.0,-0.13,0.05,10.38,10.44,4.0,0.01,1.54,5.62,X40,NTT,FMCG
20,CIPLA,1795.00,-19.78,52.0,H-LC,5.94,213926.0,9426.0,31982.0,0.64,4.61,14.95,20.25,10.0,0.29,1.64,13.74,X40N,BTT,PHARMA
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-17.86,76.0,H-SC,8.41,187816.0,-51599.0,324565.0,6.48,-21.55,172.81,114.01,133.0,-0.16,1.44,72.26,AR,ATH,IT
76,TRIDENT,48.00,-5.29,63.0,M-SC,5.90,73655.0,-18600.0,44907.0,6.08,-20.16,60.97,28.51,224.0,-0.41,0.57,26.04,XR,NTT,TEXTILES
16,BSOFT,836.99,-12.08,55.0,H-SC,16.40,67725.0,-42324.0,78751.0,3.52,-38.46,116.28,33.10,131.0,-0.54,0.52,12.54,XR,ATH,IT
72,TATAELXSI,9161.00,-19.34,57.0,H-MC,8.26,80430.0,-22222.0,47824.0,3.45,-21.65,59.46,24.94,98.0,-0.46,0.62,22.26,OX40N,NTT,IT
75,TITAGARH,1548.00,-2.98,65.0,H-SC,7.10,159110.0,-32905.0,111791.0,2.86,-17.14,70.26,41.08,158.0,-0.29,1.22,34.83,XY24,NTT,ENGINEERING


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.0,7.55,49.0,H-SC,3.70,159720.0,-40496.0,116516.0,-2.57,-20.23,72.95,37.97,144.0,-0.35,1.23,28.24,XY24,NTT,INSURANCE
82,VALIANTORG,838.0,-333.94,62.0,H-SC,8.73,99580.0,-33706.0,118301.0,-1.72,-25.29,118.80,63.47,139.0,-0.28,0.76,66.92,XR,NTT,CHEMICALS
46,LAOPALA,464.0,117.42,53.0,H-SC,3.04,69143.0,-31703.0,57527.0,-1.48,-31.44,83.20,25.61,142.0,-0.55,0.53,28.19,AR,NTT,CERAMICS
64,SHALBY,327.0,1067.46,72.0,M-SC,2.36,158752.0,-22391.0,67851.0,-0.99,-12.36,42.74,25.10,235.0,-0.33,1.22,27.22,XY24,NTT,HEALTHCARE
27,GREENPANEL,537.0,193.28,44.0,M-SC,7.12,130906.0,-52172.0,131155.0,-0.87,-28.50,100.19,43.14,230.0,-0.40,1.00,23.14,XY24,NTT,MISC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,71.0,H-SC,12.83,130734.0,-11745.0,31272.0,2.00,-8.24,23.92,13.70,163.0,-0.38,1.00,54.30,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,60.0,M-SC,5.58,88163.0,-12614.0,12704.0,-0.84,-12.52,14.41,0.09,245.0,-0.99,0.68,15.83,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,64.0,H-SC,2.36,225360.0,-44307.0,80634.0,0.70,-16.43,35.78,13.47,138.0,-0.55,1.73,13.98,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.02,140480.0,-34079.0,34109.0,0.46,-19.52,24.28,0.02,221.0,-1.00,1.08,22.19,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.05,45.0,H-SC,2.28,114138.0,-29665.0,56407.0,-0.06,-20.63,49.42,18.60,149.0,-0.53,0.88,25.95,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.76,52.0,H-LC,13.94,292114.0,-53842.0,177693.0,1.91,-15.56,60.83,35.80,1.0,-0.30,2.24,3.48,X40,BTT,IT
39,INFY,2275.00,-16.37,57.0,H-LC,10.13,325251.0,12363.0,159308.0,1.51,3.95,48.98,54.87,3.0,0.08,2.50,10.86,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200246.0,108.0,20786.0,-0.13,0.05,10.38,10.44,4.0,0.01,1.54,5.62,X40,NTT,FMCG
83,VBL,671.64,-15.68,40.0,H-LC,6.38,303688.0,-12154.0,124816.0,0.32,-3.85,41.10,35.67,5.0,-0.10,2.33,9.47,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,48.0,H-LC,12.54,247656.0,-13963.0,133165.0,0.54,-5.34,53.77,45.57,7.0,-0.10,1.90,5.38,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,2326.38,9.32,53.0,H-MC,3.54,188650.0,23980.0,43993.0,-0.03,14.56,23.32,41.28,89.0,0.55,1.45,23.71,X40N,ATH,AC
4,ANGELONE,3033.00,9.18,43.0,H-SC,6.35,193676.0,2670.0,55023.0,1.11,1.40,28.41,30.21,157.0,0.05,1.49,22.57,X40N,NTT,FINANCE
84,VOLTAS,1918.49,0.74,59.0,H-MC,3.29,211500.0,19758.0,76267.0,-0.16,10.30,36.06,50.08,99.0,0.26,1.62,17.80,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,63.0,H-MC,1.73,248882.0,1151.0,73918.0,0.52,0.46,29.70,30.30,92.0,0.02,1.91,13.82,X40,ATH,ELECTRICAL
24,DABUR,735.00,-2.22,62.0,H-MC,5.58,212764.0,15736.0,73893.0,0.24,7.99,34.73,45.49,102.0,0.21,1.63,22.07,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,66.0,H-MC,2.82,110584.0,10080.0,20558.0,0.98,10.03,18.59,30.49,79.0,0.49,0.85,45.68,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.53,60.0,M-SC,2.50,102128.0,10630.0,68119.0,-0.69,11.62,66.70,86.07,223.0,0.16,0.78,48.31,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.91,51.0,H-SC,12.17,137122.0,13786.0,119982.0,0.48,11.18,87.50,108.45,119.0,0.11,1.05,36.03,AR,ATH,MISC
86,WIPRO,420.00,-11.98,51.0,M-LC,6.75,154473.0,3528.0,105922.0,2.73,2.34,68.57,72.51,53.0,0.03,1.19,8.50,XR,NTT,IT
12,BANKINDIA,190.00,-31.68,54.0,H-MC,5.46,175532.0,-4276.0,116694.0,1.21,-2.38,66.48,62.52,88.0,-0.04,1.35,30.90,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6700.00,51.0,L-SC,6.08,74640.0,-19170.0,95786.0,-0.10,-20.43,128.33,81.67,269.0,-0.20,0.57,44.54,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.51,46.0,H-SC,14.68,88838.0,-12125.0,106659.0,1.79,-12.01,120.06,93.63,148.0,-0.11,0.68,32.32,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,42.18,90240.0,-23309.0,63358.0,0.66,-20.53,70.21,35.27,268.0,-0.37,0.69,120.86,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,60.0,H-SC,13.03,95595.0,-2385.0,23985.0,-0.14,-2.43,25.09,22.05,152.0,-0.10,0.73,38.95,XR,ATH,FINANCE
82,VALIANTORG,838.00,-333.94,62.0,H-SC,8.73,99580.0,-33706.0,118301.0,-1.72,-25.29,118.80,63.47,139.0,-0.28,0.76,66.92,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,42.18,90240.0,-23309.0,63358.0,0.66,-20.53,70.21,35.27,268.0,-0.37,0.69,120.86,XR,NTT,HOTELS
60,SAMMAANCAP,326.00,-171.59,66.0,M-SC,3.89,83100.0,-19110.0,112501.0,1.60,-18.70,135.38,91.37,208.0,-0.17,0.64,34.90,XY25,NTT,FINANCE
8,ATULAUTO,844.00,3689.19,62.0,M-SC,7.61,118118.0,-29250.0,81064.0,-0.81,-19.85,68.63,35.16,236.0,-0.36,0.91,20.65,XY24,NTT,AUTO
75,TITAGARH,1548.00,-2.98,65.0,H-SC,7.10,159110.0,-32905.0,111791.0,2.86,-17.14,70.26,41.08,158.0,-0.29,1.22,34.83,XY24,NTT,ENGINEERING
71,TANLA,1963.11,-17.86,76.0,H-SC,8.41,187816.0,-51599.0,324565.0,6.48,-21.55,172.81,114.01,133.0,-0.16,1.44,72.26,AR,ATH,IT


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,44.43
2,50,75.06


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.96
LC    35.33
MC    23.74
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.66
X40      16.08
X40N     13.15
XY25     10.29
XR        9.51
AR        8.40
OX40N     7.79
MH        1.79
X200      1.79
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.81
H-SC    25.13
H-MC    20.65
M-SC    14.16
M-LC     7.41
M-MC     2.76
L-SC     1.67
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,12.96,-15.48,73.95
FMCG,12.28,-3.71,38.86
FINANCE,9.12,-17.67,59.99
BANKS,7.70,-13.52,61.73
PAINTS,6.18,-14.71,40.38
MISC,6.02,-16.98,84.03
ELECTRICAL,5.88,-7.57,66.86
AC,3.82,2.70,30.25
AUTO,3.50,-15.04,66.91


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2729531.0,22
AR,1175721.0,9
XR,1053286.0,13
X40,871859.0,10
X40N,563844.0,10
OX40N,444962.0,10
XY25,411855.0,7
SR,196920.0,2
X5K,129173.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2748605.0,28
M-SC,1400089.0,17
H-LC,1373716.0,15
H-MC,1155328.0,15
M-LC,413184.0,5
M-MC,318139.0,2
L-SC,243275.0,3
L-MC,59614.0,1
L-LC,39933.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,912596.0,6
M-SC,XY24,819803.0,7
H-SC,AR,644241.0,5
H-LC,X40,584248.0,5
H-SC,XR,498886.0,6
H-MC,XY24,462019.0,4
H-LC,AR,371964.0,2
M-MC,XY24,318139.0,2
H-SC,X40N,244882.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
